## CNN on EMG
修订日期：22.5.18
本代码展示CNN在EMG上训练和测试的示例。所有代码与OPG_EMG 的前半部分相似，即截取了GAN训练前的部分。

本示例展示了从含有六类肌电信号的数据集中训练出六分类的 CNN，数据集来自自采数据。

In [2]:
# -*- coding:utf-8 -*-
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
# from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LogisticRegressionCV
from sklearn import metrics
# from sklearn.preprocessing import label_binarize
# from sklearn import preprocessing
# from sklearn import tree 

import torch
import torch.nn as nn
import torch.nn.init as init
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.optim as optim
import scipy.io as scio
# import hiddenlayer as h
from visdom import Visdom
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchsummary import summary
from torchviz import make_dot

import datetime
import os 

import sys
sys.path.append("..")
from utils.reuse import *
from utils.networks import *
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2
device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
cuda:0


## 预备操作
设置检查点、visdom 日志文件存储等日志性文件存储位置；
初始化 visdom,记得先在命令行输入 visdom 运行（python环境下）

In [6]:
# 以下是检查点路径
# 请在当前环境下 CMD 输入python -m visdom.server 或 visdom 启动监视器
# 数据处理现在已移至 emgDataprocess.ipynb
# 现在model_Dir 作为所有文件的父目录，不再分设开导致文件难寻
model_Dir = ".//model//opg_0217a//"
if not os.path.exists(model_Dir):
    os.makedirs(model_Dir)
# 这是正常定期检查点存储位置
ckpDir = model_Dir + "ckp//"
if not os.path.exists(ckpDir):
    os.makedirs(ckpDir)
# 这里是特挑最佳AUC的位置
ckpDir_auc = ckpDir + "auc//"
if not os.path.exists(ckpDir_auc):
    os.makedirs(ckpDir_auc)
# 这是 visdom 日志文件存储位置，留待备用
vislogDir = model_Dir + "vislog//"
if not os.path.exists(vislogDir):
    os.makedirs(vislogDir)

def get_current_time():
    return datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S")


print(get_current_time())

timeForSave = datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S")

2022_05_18_15_17_59


In [ ]:
# 以下是 visdom 监视窗口初始化，实现每次启用时重新加载，这里只写了 NameError 以防其他错误不能被发现
# 如果没有下载 visdom 需要提前下载并配置好，或者自行注释掉相关部分以避免启用
class visdom_account:
    def __init__(self):
        self.port = 8097
        self.server = "http://localhost"
        self.base_url = "/"
        self.username = "admin"
        self.passward = "1234"
        self.evns = "train"


viz_acnt = visdom_account()


def viz_init():
    try:
        viz
    except NameError:
        viz = Visdom(
            env=viz_acnt.evns, log_to_filename=vislogDir + "vislog_" + timeForSave
        )
        print("visdom has started")
    else:
        viz.close()
        del viz
        print("last visdom session closed")
        viz = Visdom(
            env=viz_acnt.evns, log_to_filename=vislogDir + "vislog_" + timeForSave
        )
        print("visdom has restarted")
    return viz


viz = viz_init()

## 定义神经网络结构
CNN 训练部分，为了获得一个已知类分类器

In [8]:
# 自定义神经网络,CNN
def get_num_correct(preds, labels):
    return preds.argmax(dim=1).eq(labels).sum().item()


class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(
            in_channels=32, out_channels=32, kernel_size=3, padding=1
        )
        self.conv3 = nn.Conv2d(
            in_channels=32, out_channels=32, kernel_size=3, padding=0
        )

        self.fc1 = nn.Linear(in_features=32 * 4 * 198, out_features=128)
        self.out = nn.Linear(in_features=128, out_features=6)
        self.dr1 = nn.Dropout2d(0.2)

    def forward(self, t):
        # (1) input layer
        t = t

        # (2) hidden conv layer
        t = self.conv1(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=1)

        # (3) hidden conv layer
        t = self.conv2(t)
        t = F.relu(t)
        # t = self.dr1(t)
        t = F.max_pool2d(t, kernel_size=2, stride=1)

        # (4) hidden linear layer
        t = t.reshape(-1, 32 * 4 * 198)
        t = self.fc1(t)
        t = F.relu(t)
        t = self.dr1(t)

        # (5) output layer
        t = self.out(t)

        return t


net = Network()
# 打印网络，检查输入输出 shape是否正确
print(net)
summary(net, (1, 200, 6), batch_size=1, device="cpu")
# 可视化结构，torchviz
sampleInput = torch.randn(1, 1, 200, 6).requires_grad_(True)
sampleOutput = net(sampleInput)
print("Outputshape:", sampleOutput.shape)

Network(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=25344, out_features=128, bias=True)
  (out): Linear(in_features=128, out_features=6, bias=True)
  (dr1): Dropout2d(p=0.2, inplace=False)
)
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [1, 32, 200, 6]             320
            Conv2d-2            [1, 32, 199, 5]           9,248
            Linear-3                   [1, 128]       3,244,160
         Dropout2d-4                   [1, 128]               0
            Linear-5                     [1, 6]             774
Total params: 3,254,502
Trainable params: 3,254,502
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.0

## 数据集加载、构建

In [11]:
# CNN训练数据加载
transform = transforms.Compose([
    transforms.ToTensor(),  # 将图片转换为Tensor,归一化至[0,1]
])

class EMGDataset(Dataset):
 
    def __init__(self, data, label):
        self.data = data
        self.label = label
        self.transforms = transform
 
    def __getitem__(self, index):
        emgData = self.data[index,:,:,:]
        emgData = np.squeeze(emgData)
        emglabel = self.label[index]
        emglabel = emglabel.astype(np.int16)
        emgData = self.transforms(emgData)
        # 一维数据用下面的这个就行
        # emgData = torch.Tensor(emgData)     
        return emgData,emglabel
 
    def __len__(self):
        return len(self.label)
 
 
# if __name__ == '__main__':
dataarray = np.load('../../data/OpenganDataSet_220217a_0226.npy',allow_pickle=True)
CNNdataset = dataarray.item()
print(type(CNNdataset))
traindata = CNNdataset['Xtrain']
trainlabel = CNNdataset['Ytrain']
testdata = CNNdataset['Xtest']
testlabel = CNNdataset['Ytest']
valdata = CNNdataset['Xval']
vallabel = CNNdataset['Yval']

trainlabel = trainlabel[:,0]
testlabel = testlabel[:,0]
vallabel = vallabel[:,0]
# trainunknownc_label = trainunknownc_label[:,0]
# print(type(trainlabel))
train_set = EMGDataset(traindata, trainlabel)
test_set = EMGDataset(testdata, testlabel)
val_set = EMGDataset(valdata, vallabel)
# train_unknown = EMGDataset(trainunknown_data,trainunknownc_label)
# train_loader = torch.utils.data.DataLoader(train_set, batch_size=1, shuffle=True, pin_memory=True,
#                                             num_workers=3)

# 试运行数据集，构建迭代器
sample = iter(test_set)
print(sample)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=2, shuffle=False)
batch1 = iter(train_loader)

<class 'dict'>


In [14]:
batch2 = next(batch1)
d_x,d_y = batch2
print(
    "x:",
    d_x.shape,
    d_x,
    "\n",
    "y:",
    d_y.shape,
    d_y
)

x: torch.Size([2, 1, 200, 6]) tensor([[[[ 0.0560,  0.0085,  0.0103, -0.0063,  0.0692, -0.0128],
          [ 0.0630,  0.0048,  0.0117, -0.0094,  0.0801,  0.0112],
          [ 0.0523,  0.0097,  0.0128, -0.0087,  0.0571,  0.0216],
          ...,
          [ 0.0189,  0.0077, -0.0629,  0.1049,  0.1224, -0.0539],
          [ 0.0183,  0.0043, -0.0636,  0.0571,  0.1217, -0.0036],
          [ 0.0197, -0.0033, -0.0216, -0.0099,  0.0616,  0.0214]]],


        [[[-0.0192,  0.0187, -0.0082,  0.0171, -0.1246,  0.0031],
          [-0.0205,  0.0269, -0.0022, -0.0093, -0.0880, -0.0135],
          [-0.0192,  0.0206,  0.0065, -0.0338, -0.0170,  0.0189],
          ...,
          [-0.0324, -0.0698,  0.0012,  0.0213, -0.0666,  0.0129],
          [-0.0182, -0.0770, -0.0173,  0.0614, -0.0370,  0.0245],
          [ 0.0020, -0.0550, -0.0124,  0.0411, -0.0115,  0.0102]]]],
       dtype=torch.float64) 
 y: torch.Size([2]) tensor([0, 0], dtype=torch.int16)


In [6]:
# CNN 训练
# 损失
# criterion = nn.BCEWithLogitsLoss().to(device)
# 加载数据，设置优化器
train_loader = torch.utils.data.DataLoader(train_set, batch_size=256, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=100, shuffle=True)
optimizer = torch.optim.Adam(net.parameters(), lr=0.0002)
lr_schedule = torch.optim.lr_scheduler.StepLR(optimizer, 100, gamma=0.5, last_epoch=-1)
# 初始化 visdom
viz.close()
viz = viz_init()
if not os.path.exists(vislogDir):
    os.makedirs(vislogDir)
viz = Visdom(
    env=viz_acnt.evns, log_to_filename=vislogDir + "vislog_" + get_current_time()
)
vizx = 0
viz.text(
    "MONITOR: Show train process~~",
    win="Monitor",
    opts={
        "title": "ProcessMonitor",
    },
)

total_test_acc = 0
total_test_correct = 0
totaltest = 0
# 训练过程
epoch_num = 600
net.to(device)
for epoch in range(epoch_num):
    viz.text(
        "ep" + str(epoch + 1) + " start",
        win="Monitor",
        opts={
            "title": "ProcessMonitor",
        },
    )
    total_loss = 0
    total_correct = 0
    curr_total_correct = 0
    total_traintnum = 0
    for batch in train_loader:
        # 载入本批次数据
        images, labels = batch
        images = images.to(torch.float32)
        labels = labels.long()
        net.train()
        preds = net(images.to(device))
        trainloss = F.cross_entropy(preds.to(device), labels.to(device))  # 真实数据的鉴别器损失
        optimizer.zero_grad()
        trainloss.backward()  # Calculate Gradients
        optimizer.step()  # Update Weight
        total_loss += trainloss.item()
        curr_total_correct = get_num_correct(preds.to(device), labels.to(device))
        total_correct += curr_total_correct
        total_traintnum += labels.size(0)
    total_train_acc = total_correct / total_traintnum
    total_correct = 0
    # 测试
    net.eval()
    total_testnum = 0
    for testemgdatas, testemglabels in test_loader:  # Get Batch
        testemgdatas = testemgdatas.to(torch.float32)
        testemglabels = testemglabels.long()
        predstest = net(testemgdatas.to(device))
        testloss = F.cross_entropy(
            predstest.to(device), testemglabels.to(device)
        )  # Calculate Loss
        curr_test_correct = get_num_correct(
            predstest.to(device), testemglabels.to(device)
        )
        total_testnum += testemglabels.size(0)
        total_test_correct += curr_test_correct
        # totaltest += testemglabels.size(0)
    # total_test_acc = total_test_correct/(trainlabel.size)
    total_test_acc = total_test_correct / total_testnum
    print(
        "epoch",
        epoch,
        "total_train_acc:",
        total_train_acc,
        "loss:",
        total_loss,
        "test_loss:",
        float(testloss),
        "test_acc:",
        total_test_acc,
    )
    total_test_correct = 0
    # 可视化，每 epoch 更新
    viz.line(
        [float(trainloss)],
        [epoch],
        win="loss_perEpoch",
        name="G_loss",
        update="append",
        opts=dict(title="loss_perEpoch", xlabel="epoch", ylabel="loss"),
    )
    viz.line(
        [float(testloss)], [epoch], win="loss_perEpoch", name="D_loss", update="append"
    )
    viz.line(
        [float(total_train_acc)],
        [epoch],
        win="acc_perEpoch",
        name="train_acc",
        update="append",
        opts=dict(title="acc_perEpoch", xlabel="epoch", ylabel="acc"),
    )
    viz.line(
        [float(total_test_acc)],
        [epoch],
        win="acc_perEpoch",
        name="test_acc",
        update="append",
    )

    viz.line(
        [float(optimizer.state_dict()["param_groups"][0]["lr"])],
        [epoch],
        win="lr_perEpoch",
        name="G_loss",
        update="append",
        opts=dict(title="lr_perEpoch", xlabel="epoch", ylabel="lr"),
    )
    # 更新学习率
    lr_schedule.step()
    # viz.text('updating weights',win='Monitor',append=True,opts = {'title':'ProcessMonitor',},)
    # 定期保存
    if (epoch + 1) % 100 == 0:
        timeForSave = datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
        checkpointPath = (
            ckpDir
            + "c_ep_"
            + str(epoch + 1)
            + "_acc"
            + str(int(total_test_acc * 10000))
            + "_"
            + timeForSave
            + ".pth"
        )
        c_state = {
            "model": net.state_dict(),
            "optimizer": optimizer.state_dict(),
            "epoch": epoch,
        }
        torch.save(c_state, checkpointPath)
        viz.text(
            "epoch " + str(epoch + 1) + " model saved",
            win="Monitor",
            append=True,
            opts={
                "title": "ProcessMonitor",
            },
        )


checkpointPath_model = (
    model_Dir
    + "c_final_"
    + "acc"
    + str(int(total_test_acc * 10000))
    + "_"
    + timeForSave
    + ".pth"
)
torch.save(net.state_dict(), checkpointPath_model)

Setting up a new session...
Setting up a new session...


visdom has started
epoch 0 total_train_acc: 0.20222434824204735 loss: 59.06453335285187 test_loss: 1.7725965976715088 test_acc: 0.325619384699156
epoch 1 total_train_acc: 0.26201865582396555 loss: 57.75129568576813 test_loss: 1.7197678089141846 test_acc: 0.24802613667301934
epoch 2 total_train_acc: 0.35745037072470703 loss: 53.50155305862427 test_loss: 1.524469017982483 test_acc: 0.4353389599782194
epoch 3 total_train_acc: 0.41664673523080603 loss: 47.39896774291992 test_loss: 1.2795482873916626 test_acc: 0.5085760958344677
epoch 4 total_train_acc: 0.4869648409471418 loss: 42.70749843120575 test_loss: 1.3570644855499268 test_acc: 0.6496052273346039
epoch 5 total_train_acc: 0.5520210475962688 loss: 38.62566924095154 test_loss: 1.0890090465545654 test_acc: 0.7032398584263545
epoch 6 total_train_acc: 0.6144463047117914 loss: 35.16096884012222 test_loss: 1.2244209051132202 test_acc: 0.7258371903076504
epoch 7 total_train_acc: 0.6645539344654389 loss: 31.741951048374176 test_loss: 0.9727004

KeyboardInterrupt: 